## Investigating Bids

The purpose of this workbook is to investigate bids in the country of saudi arabia to determine how much money is spent on different types of projects


In [1]:
import pandas as pd
import os
import re
# List the files in the csv subdirectory of the ministry
files_path = "opendata/ministry_of_finance/csv"
csv_files = os.listdir(files_path)

filtered_files = [file for file in csv_files if re.search(r'Government\sContract\sPayment', file, re.IGNORECASE)]

print(filtered_files)


['Government Contract Payment January 2024 CSV.csv', 'Government Contract Payment may 2024 CSV.csv', 'Government Contract Payment July 2024 CSV.csv', 'Government Contract Payment June 2024 CSV.csv', 'Government Contract Payment February 2024CSV.csv']


In [ ]:

from openai import OpenAI
from dotenv import load_dotenv
from unicodedata import category
import numpy as np

load_dotenv('examples/workbooks/.env')

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Create a function for openai to translate text from arabia to english
def translate_text(text):
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that translates text from Arabic to English. Whatever you return, it must be in english and camel case."},
            {"role": "user", "content": f"{text}"}
        ],
        max_tokens=50,
        temperature=0.0
    )
    return response.choices[0].message.content


budget_1 = pd.read_csv(f"{files_path}/{filtered_files[0]}")
# Create an index of the columns that are arabic and the ones that are english. 1 = arabic, 0 = english 
language_index = np.array([1 if re.search(r'[ع-ي]', col) else 0 for col in budget_1.columns])
print(language_index)
print("Total number of contracts: ", budget_1.shape[0])

# Before attempting to translate the values in a pandas series from arabic to english, we need to check if the values are in the arabic alphabet
# If they are, we can translate them, if not, we can leave them as they are

# Create a function to check if a value is in the arabic alphabet
def is_arabic(value):
    return 1 if re.search(r'[ع-ي]', value) else 0

#print(budget_1.head())
english_version = budget_1.map(lambda x: translate_text(x) if isinstance(x, str) and is_arabic(x) else x)
english_version.columns = [translate_text(i) for i in english_version.columns]



[1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Total number of contracts:  10
  تاريخ الإنشاء  Unnamed: 1                          التصنيف الاقتصادي  \
0     1/28/2024         NaN  قطع غيار الأجهزة ولآلات والمعدات والمضخات   
1     1/10/2024         NaN                             مصروفات مكتبية   
2     1/14/2024         NaN                              حفلات وضيافات   
3     1/30/2024         NaN                  مواد ومستلزماتصناعية أخرى   
4     1/30/2024         NaN                  مواد ومستلزماتصناعية أخرى   

      التصنيف الصناعي                                          وصف مختصر  \
0            غير مصنف  صيانة آلات تصوير المستندات الموجودة بمختلف إدا...   
1      المرافق الصحية  تامين مستلزمات طبية لطب الاسنان  من السوق الال...   
2            غير محدد                                          غير متوفر   
3  محطات تحلية المياه  توريد مواد كيماويات للمؤسسة العامة لتحلية المي...   
4  محطات تحلية المياه  توريد المواد وقطع الغيار كيماويات المؤسسة العا...   

  وحدة المدة  مدة العقد حالة ال

In [5]:
budget_2 = pd.read_csv(f"{files_path}/{filtered_files[1]}")
# Create an index of the columns that are arabic and the ones that are english. 1 = arabic, 0 = english 
language_index = np.array([1 if re.search(r'[ع-ي]', col) else 0 for col in budget_2.columns])
print(language_index)
print("Total number of contracts: ", budget_2.shape[0])
budget_2



[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Total number of contracts:  12


,تاريخ الإنشاء,التصنيف الصناعي,وصف مختصر,وحدة المدة,مدة العقد,حالة العقد,تصنيف العقد,نوع العقد / التعميد,اسم العقد,المورد,السجل التجاري,رقم العقد,التنظيم الإداري,القطاع,الجهة الرئيسية
0,5/21/2024,المرافق الصحية,تامين مستلزمات قسم التاهيل لمستشفى الدرعية رقم...,يوم,20,معتمد,توريد,تعميد,تامين مستلزمات قسم التاهيل لمستشفى الدرعية رقم...,الشركة الوطنية للشراء الموحد للادوية والاجهزة ...,1010248277,1418877,تجمع الرياض الصحي الثالث,قطاع الصحة و التنمية الإجتماعية,وزارة الصحة
1,5/30/2024,المرافق الصحية,تامين بنود طبية لزوم تجمع القصيم الصحي,يوم,59,معتمد,توريد,تعميد,تامين بنود طبية لزوم تجمع القصيم الصحي,الشركة الوطنية للشراء الموحد للادوية والاجهزة ...,1010248277,1427913,التجمع الصحي بمنطقة القصيم,قطاع الصحة و التنمية الإجتماعية,وزارة الصحة
2,5/22/2024,غير محدد,غير متوفر,يوم,120,معتمد,غير محدد,تعميد,تامين مستلزمات طبية,الشركة الوطنية للشراء الموحد للادوية والاجهزة ...,1010248277,1420006,جامعة تبوك,قطاع التعليم,وزارة التعليم العالي
3,5/14/2024,المرافق الصحية,توريد جزء من طلبية الأدوية السنوية للمدينة الطبية,يوم,65,معتمد,توريد,تعميد,تأمين الطلبية السنوية للعيادات الجامعية لطب ال...,الشركة الوطنية للشراء الموحد للادوية والاجهزة ...,1010248277,1413285,جامعة الملك خالد,قطاع التعليم,وزارة التعليم العالي
4,5/28/2024,غير محدد,غير متوفر,يوم,120,معتمد,غير محدد,تعميد,تامين ادوية,الشركة الوطنية للشراء الموحد للادوية والاجهزة ...,1010248277,1425124,جامعة تبوك,قطاع التعليم,وزارة التعليم العالي
5,5/14/2024,المرافق التعليمية,غير متوفر,يوم,15,معتمد,توريد,تعميد,تأمين احتياج كلية العلوم الطبية التطبيقية من ا...,الشركة الوطنية للشراء الموحد للادوية والاجهزة ...,1010248277,1414187,جامعة الأمير سطام بن عبدالعزيز,قطاع التعليم,وزارة التعليم العالي
6,5/14/2024,المرافق الصحية,تأمين الطلبية السنوية للعيادات الجامعية لطب ال...,يوم,55,معتمد,توريد,تعميد,تأمين الطلبية السنوية للعيادات الجامعية لطب ال...,الشركة الوطنية للشراء الموحد للادوية والاجهزة ...,1010248277,1413423,جامعة الملك خالد,قطاع التعليم,وزارة التعليم العالي
7,5/20/2024,المرافق التعليمية,شراء الادوية,يوم,60,معتمد,توريد,تعميد,شراء ادوية,الشركة الوطنية للشراء الموحد للادوية والاجهزة ...,1010248277,1418003,جامعة بيشة,قطاع التعليم,وزارة التعليم العالي
8,5/28/2024,المرافق الصحية,تأمين مستلزمات الخدمات التعليمية -نوبكو- رقم ا...,يوم,30,معتمد,توريد,تعميد,تأمين مستلزمات الخدمات التعليمية -نوبكو- رقم ا...,الشركة الوطنية للشراء الموحد للادوية والاجهزة ...,1010248277,1425134,جامعة الملك عبدالعزيز,قطاع التعليم,وزارة التعليم العالي
9,5/30/2024,المرافق الصحية,تأمين مستلزمات الخدمات التعليمية -نوبكو- رقم ا...,يوم,40,معتمد,توريد,تعميد,تأمين مستلزمات الخدمات التعليمية -نوبكو- رقم ا...,الشركة الوطنية للشراء الموحد للادوية والاجهزة ...,1010248277,1427810,جامعة الملك عبدالعزيز,قطاع التعليم,وزارة التعليم العالي


In [6]:
english_version2 = budget_2.map(lambda x: translate_text(x) if isinstance(x, str) and is_arabic(x) else x)
english_version2.columns = [translate_text(i) for i in english_version2.columns]
english_version2


,CreationDate,IndustrialClassification,BriefDescription,DurationUnit,ContractDuration,ContractStatus,NodeClassification,TypeOfContract / Commissioning,ContractName,TheSupplier,CommercialRegister,ContractNumber,AdministrativeOrganization,TheSector,MainEntity
0,5/21/2024,SanitaryFacilities,VitaminSuppliesForTheRehabilitationDepartmentA...,Day,20,Certified,Supply,Baptism,VitaminSuppliesForTheRehabilitationDepartmentA...,NationalCompanyForUnifiedPurchasingOfMedicines...,1010248277,1418877,RiyadhHealthClusterThree,HealthAndSocialDevelopmentSector,MinistryOfHealth
1,5/30/2024,HealthFacilities,MedicalVitaminItemsForTheQassimHealthGathering,Day,59,Certified,Supply,Baptism,VitaminMedicalItemsForQassimHealthGathering,NationalCompanyForUnifiedPurchasingOfMedicines...,1010248277,1427913,TheHealthGatheringInQassimRegion,HealthAndSocialDevelopmentSector,MinistryOfHealth
2,5/22/2024,Unspecified,NotAvailable,Day,120,Certified,Unspecified,Baptism,VitaminsAndMedicalSupplies,NationalCompanyForUnifiedPurchasingOfMedicines...,1010248277,1420006,TabukUniversity,EducationSector,MinistryOfHigherEducation
3,5/14/2024,HealthFacilities,SupplyingPartOfTheAnnualMedicalSuppliesOrderFo...,Day,65,Certified,Supply,Baptism,AnnualOrderInsuranceForUniversityClinicsOfFami...,NationalCompanyForUnifiedPurchasingOfMedicines...,1010248277,1413285,KingKhalidUniversity,EducationSector,MinistryOfHigherEducation
4,5/28/2024,Unspecified,NotAvailable,Day,120,Certified,Unspecified,Baptism,VitaminMedications,NationalCompanyForUnifiedPurchasingOfMedicines...,1010248277,1425124,TabukUniversity,EducationSector,MinistryOfHigherEducation
5,5/14/2024,EducationalFacilities,NotAvailable,Day,15,Certified,Supply,Baptism,Nupco'sProvisionOfTheCollegeOfMedicalAppliedSc...,NationalCompanyForUnifiedPurchasingOfMedicines...,1010248277,1414187,PrinceSattamBinAbdulazizUniversity,EducationSector,MinistryOfHigherEducation
6,5/14/2024,HealthFacilities,AnnualOrderInsuranceForUniversityClinicsOfFami...,Day,55,Certified,Supply,Baptism,AnnualOrderInsuranceForUniversityClinicsOfFami...,NationalCompanyForUnifiedPurchasingOfMedicines...,1010248277,1413423,KingKhalidUniversity,EducationSector,MinistryOfHigherEducation
7,5/20/2024,EducationalFacilities,PurchasingMedicines,Day,60,Certified,Supply,Baptism,PurchasingMedications,NationalCompanyForUnifiedPurchasingOfMedicines...,1010248277,1418003,BishaUniversity,EducationSector,MinistryOfHigherEducation
8,5/28/2024,SanitaryFacilities,InsuranceOfEducationalServicesSupplies -Nubco-...,Day,30,Certified,Supply,Baptism,InsuranceOfEducationalServicesSupplies -Nubco-...,NationalCompanyForUnifiedPurchasingOfMedicines...,1010248277,1425134,KingAbdulazizUniversity,EducationSector,MinistryOfHigherEducation
9,5/30/2024,HealthFacilities,InsuranceOfEducationalServicesSupplies -Nubco-...,Day,40,Certified,Supply,Baptism,EducationalServicesSuppliesInsurance -Nubco- R...,NationalCompanyForUnifiedPurchasingOfMedicines...,1010248277,1427810,KingAbdulazizUniversity,EducationSector,MinistryOfHigherEducation


In [8]:
#I want to create a function that will allow me to save this file with the same naming 
# convention as the original file but with the english version of the column names into a subdirectory called "english"
#def save_english_version(df, filename):
#    df.to_csv(f"english/{filename}", index=False)

#save_english_version(english_version, filtered_files[0])
#save_english_version(english_version2, filtered_files[1])
#Count the number of contracts by mainentity
english_version2['MainEntity'].value_counts()



MainEntity
MinistryOfHigherEducation    10
MinistryOfHealth              2
Name: count, dtype: int64